<a href="https://www.kaggle.com/code/minanalminazor/taylor-swift-reputation-album-analyze?scriptVersionId=140680865" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import string
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/taylor-swift-song-lyrics-all-albums/06-reputation.csv
/kaggle/input/taylor-swift-song-lyrics-all-albums/01-taylor_swift.csv
/kaggle/input/taylor-swift-song-lyrics-all-albums/05-1989_deluxe.csv
/kaggle/input/taylor-swift-song-lyrics-all-albums/08-folklore_deluxe_version.csv
/kaggle/input/taylor-swift-song-lyrics-all-albums/09-evermore_deluxe_version.csv
/kaggle/input/taylor-swift-song-lyrics-all-albums/07-lover.csv
/kaggle/input/taylor-swift-song-lyrics-all-albums/02-fearless_taylors_version.csv
/kaggle/input/taylor-swift-song-lyrics-all-albums/04-red_deluxe_edition.csv
/kaggle/input/taylor-swift-song-lyrics-all-albums/03-speak_now_deluxe_package.csv


/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# load dataset
reputation = pd.read_csv("../input/taylor-swift-song-lyrics-all-albums/06-reputation.csv")
reputation

,album_name,track_title,track_n,lyric,line
0,reputation,...Ready for It?,1,Knew he was a killer first time that I saw him,1
1,reputation,...Ready for It?,1,Wondered how many girls he had loved and left ...,2
2,reputation,...Ready for It?,1,"But if he's a ghost, then I can be a phantom",3
3,reputation,...Ready for It?,1,"Holdin' him for ransom, some",4
4,reputation,...Ready for It?,1,"Some boys are tryin' too hard, he don't try at...",5
...,...,...,...,...,...
983,reputation,New Year’s Day,15,"(Hold on to the memories, they will hold on to...",40
984,reputation,New Year’s Day,15,Please don't ever become a stranger,41
985,reputation,New Year’s Day,15,"(Hold on to the memories, they will hold on to...",42
986,reputation,New Year’s Day,15,Whose laugh I could recognize anywhere,43


In [3]:
#get info about the DataFrame
reputation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   album_name   988 non-null    object
 1   track_title  988 non-null    object
 2   track_n      988 non-null    int64 
 3   lyric        988 non-null    object
 4   line         988 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 38.7+ KB


In [4]:
#lowercase & remove punctuation
reputation['clean_lyric'] = reputation['lyric'].str.lower()
reputation['clean_lyric'] = reputation['clean_lyric'].str.replace('[^\w\s]','',regex=True)

In [5]:
#remove stopwords (see the next cell for illustration)
#create a small list of English stop words, feel free to edit this list
stop = ['the', 'a', 'this', 'that', 'to', 'is', 'am', 'was', 'were', 'be', 'being', 'been']

#there are three steps in one here - explained below
#we make a list of words with `.split()`
#then we remove all the words in our list
#then we join the words back together into a string
reputation['clean_lyric'] = reputation['clean_lyric'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [6]:
#create a new column to reflect if the lyrics contain midnight
reputation['midnight'] = reputation['clean_lyric'].str.contains('midnight')

In [7]:
#night, day, and other time-related words
night = ['night','midnight', 'dawn', 'dusk', 'evening', 'late', 'dark', '1am', '2am', '3am', '4am']
day = ['day', 'morning', 'light', 'sun', 'dawn', 'noon', 'golden', 'bright']
time = ['today', 'tomorrow', 'yesterday']

In [8]:
#create a regular expression string for each list of words
night = '|'.join(night)
day='|'.join(day)
time='|'.join(time)
#create a new column for each category of words
reputation['night'] = reputation['clean_lyric'].str.contains(night)
reputation['day'] = reputation['clean_lyric'].str.contains(day)
reputation['time'] = reputation['clean_lyric'].str.contains(time)
#count the number of times each category of word appears in the lyrics
sum_night = sum(reputation['night'])
sum_day = sum(reputation['day'])
sum_time = sum(reputation['time'])
#print the count of each word category
print('Night: {}'.format(sum_night))
print('Day: {}'.format(sum_day))
print('Time: {}'.format(sum_time))

Night: 25
Day: 25
Time: 0


In [9]:
#run this cell to tokenize the words in the clean_lyric column
reputation['lyrics_tok'] = reputation['clean_lyric'].str.split(' ')

In [10]:
#determine what words overall are the most frequently used words
#create a list of all the words in the lyrics_tok column
word_list = [word for list_ in reputation['lyrics_tok'] for word in list_]

#use the counter function to count the number of times each word appears
counter = collections.Counter(word_list)

#sort the word frequencies to find out the most common words she's used. 
counter = sorted(counter.items(), key=lambda x: x[1], reverse=True) 
#call the word frequency
counter[0]

('you', 325)

In [11]:
#run this cell to add a package from NLTK for our sentiment analyzer.
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [12]:
#run this cell to see how the sentiment analyzer works
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("I love Taylor Swift!")

{'neg': 0.0, 'neu': 0.137, 'pos': 0.863, 'compound': 0.7424}

In [13]:
#create a new column called polarity and apply the sia method to the clean_lyric column with a lambda expression
reputation['polarity'] = reputation['clean_lyric'].apply(lambda x:sia.polarity_scores(x))

In [14]:
#run this cell to transform the polarity dictionary into columns of the DataFrame
reputation[['neg', 'neu', 'pos', 'compound']] = reputation['polarity'].apply(pd.Series)
reputation.drop('polarity', axis=1)

,album_name,track_title,track_n,lyric,line,clean_lyric,midnight,night,day,time,lyrics_tok,neg,neu,pos,compound
0,reputation,...Ready for It?,1,Knew he was a killer first time that I saw him,1,knew he killer first time i saw him,False,False,False,False,"[knew, he, killer, first, time, i, saw, him]",0.417,0.583,0.000,-0.6486
1,reputation,...Ready for It?,1,Wondered how many girls he had loved and left ...,2,wondered how many girls he had loved and left ...,False,False,False,False,"[wondered, how, many, girls, he, had, loved, a...",0.207,0.533,0.260,0.2023
2,reputation,...Ready for It?,1,"But if he's a ghost, then I can be a phantom",3,but if hes ghost then i can phantom,False,False,False,False,"[but, if, hes, ghost, then, i, can, phantom]",0.330,0.670,0.000,-0.4497
3,reputation,...Ready for It?,1,"Holdin' him for ransom, some",4,holdin him for ransom some,False,False,False,False,"[holdin, him, for, ransom, some]",0.000,1.000,0.000,0.0000
4,reputation,...Ready for It?,1,"Some boys are tryin' too hard, he don't try at...",5,some boys are tryin too hard he dont try at al...,False,False,False,False,"[some, boys, are, tryin, too, hard, he, dont, ...",0.113,0.887,0.000,-0.1027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,reputation,New Year’s Day,15,"(Hold on to the memories, they will hold on to...",40,hold on memories they will hold on you,False,False,False,False,"[hold, on, memories, they, will, hold, on, you]",0.000,1.000,0.000,0.0000
984,reputation,New Year’s Day,15,Please don't ever become a stranger,41,please dont ever become stranger,False,False,False,False,"[please, dont, ever, become, stranger]",0.000,0.635,0.365,0.3182
985,reputation,New Year’s Day,15,"(Hold on to the memories, they will hold on to...",42,hold on memories they will hold on you,False,False,False,False,"[hold, on, memories, they, will, hold, on, you]",0.000,1.000,0.000,0.0000
986,reputation,New Year’s Day,15,Whose laugh I could recognize anywhere,43,whose laugh i could recognize anywhere,False,False,False,False,"[whose, laugh, i, could, recognize, anywhere]",0.000,0.526,0.474,0.5574


In [15]:
#calculate overall sentiment for pos, neg, sentiment
pos = reputation['pos'].sum()
neg = reputation['neg'].sum()
com = reputation['compound'].sum()
#print the overall sentiments
print("Positive: {}".format(pos))
print("Negative: {}".format(neg))
print("Compound: {}".format(com))

Positive: 122.418
Negative: 76.91999999999999
Compound: 38.720200000000006


In [16]:
#create a DataFrame filtered for only night mentions
night = reputation[reputation['night']==True]
#create a DataFrame filtered for only day mentions
day = reputation[reputation['day']==True]
midnight = reputation[reputation['midnight']==True]
#print the length of the night and day DataFrames
print(len(night))
print(len(day))
print(len(midnight))

25
25
3


In [17]:
#calculate the sentiment of each day and night DataFrame from the compound values
day_sentiment = day['compound'].mean()
night_sentiment = night['compound'].mean()
midnight_sentiment = midnight['compound'].mean()

#print the results
print("Day Sentiment:", day_sentiment)
print("Night Sentiment:", night_sentiment)
print("Midnight Sentiment:", midnight_sentiment)

Day Sentiment: 0.030452000000000007
Night Sentiment: 0.10352800000000001
Midnight Sentiment: 0.0772
